# 🎯 **ASL GESTURE RECOGNITION - NEW WEBCAM DATASET TRAINING**

**Optimized for 600 high-quality webcam images**
- Dataset: Dataset_new/ (60 images per gesture × 10 gestures)
- Domain: Real webcam conditions (perfect match for inference)
- Strategy: Aggressive augmentation + Transfer learning + Strong regularization


In [1]:
# 📦 IMPORTS
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Flatten, Dropout, BatchNormalization, 
    Conv2D, MaxPool2D, GlobalAveragePooling2D, Input
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

print("📚 Libraries imported successfully!")
print(f"🔥 TensorFlow version: {tf.__version__}")


📚 Libraries imported successfully!
🔥 TensorFlow version: 2.12.0


In [2]:
# 📁 DATASET CONFIGURATION & ANALYSIS
DATASET_PATH = 'Dataset_new'  # New webcam dataset
IMG_SIZE = 224  # Optimal for MobileNetV2 (upscale from 240x240 ROI)
BATCH_SIZE = 16  # Smaller batch for limited data
NUM_CLASSES = 10

# 📊 Check dataset structure
print("📊 DATASET ANALYSIS:")
total_images = 0
for i in range(NUM_CLASSES):
    class_path = os.path.join(DATASET_PATH, str(i))
    if os.path.exists(class_path):
        count = len([f for f in os.listdir(class_path) if f.endswith('.jpg')])
        total_images += count
        print(f"   Class {i}: {count} images")
    else:
        print(f"   ❌ Class {i}: Folder missing")

print(f"\n✅ Total images: {total_images}")
print(f"📏 Target image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"🎯 Batch size: {BATCH_SIZE}")


📊 DATASET ANALYSIS:
   Class 0: 60 images
   Class 1: 60 images
   Class 2: 60 images
   Class 3: 60 images
   Class 4: 60 images
   Class 5: 60 images
   Class 6: 60 images
   Class 7: 60 images
   Class 8: 60 images
   Class 9: 60 images

✅ Total images: 600
📏 Target image size: 224x224
🎯 Batch size: 16


In [5]:
# 🎨 AGGRESSIVE DATA AUGMENTATION (Optimized for small dataset)
train_datagen = ImageDataGenerator(
    # Normalization
    rescale=1./255,
    
    # Geometric augmentation
    rotation_range=25,          # Increased from 20
    width_shift_range=0.15,     # Hand position variation
    height_shift_range=0.15,    # Hand position variation  
    zoom_range=0.25,            # Increased from 0.2
    horizontal_flip=False,      # Don't flip - changes gesture meaning
    
    # Photometric augmentation
    brightness_range=[0.8, 1.2],  # Lighting variation
    channel_shift_range=20,       # Color variation
    
    # Advanced augmentation
    shear_range=10,              # Slight perspective change
    fill_mode='constant',        # Black fill (matches ROI background)
    cval=0,                      # Black color value
    
    # Validation split
    validation_split=0.2         # 80% train, 20% validation
)

# 📊 Validation data (no augmentation)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

print("🎨 Data augmentation configured:")
print("   ✅ Aggressive geometric transformations")
print("   ✅ Photometric variations")
print("   ✅ 80/20 train/validation split")
print("   ✅ No horizontal flip (preserves gesture meaning)")


🎨 Data augmentation configured:
   ✅ Aggressive geometric transformations
   ✅ Photometric variations
   ✅ 80/20 train/validation split
   ✅ No horizontal flip (preserves gesture meaning)


In [6]:
# 📂 DATA LOADING
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical', 
    subset='validation',
    shuffle=False,
    seed=42
)

print("📂 Data generators created:")
print(f"   🏋️ Training samples: {train_generator.samples}")
print(f"   ✅ Validation samples: {validation_generator.samples}")
print(f"   🎯 Classes found: {len(train_generator.class_indices)}")
print(f"   📊 Class mapping: {train_generator.class_indices}")


Found 480 images belonging to 10 classes.


Found 120 images belonging to 10 classes.
📂 Data generators created:
   🏋️ Training samples: 480
   ✅ Validation samples: 120
   🎯 Classes found: 10
   📊 Class mapping: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


In [7]:
# 🏗️ IMPROVED CUSTOM CNN (Optimized for small dataset)
def create_improved_cnn():
    """Enhanced CNN with strong regularization for small datasets"""
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Dropout(0.25),
        
        # Block 2  
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(), 
        GlobalAveragePooling2D(),  # Better than Flatten for small datasets
        
        # Dense layers with heavy regularization
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

# Create improved CNN
custom_cnn = create_improved_cnn()
print("🏗️ Improved Custom CNN created with:")
print("   ✅ Batch Normalization for stability")
print("   ✅ Heavy Dropout (0.25, 0.5) for regularization") 
print("   ✅ Global Average Pooling vs Flatten")
print("   ✅ Optimized for small dataset")


🏗️ Improved Custom CNN created with:
   ✅ Batch Normalization for stability
   ✅ Heavy Dropout (0.25, 0.5) for regularization
   ✅ Global Average Pooling vs Flatten
   ✅ Optimized for small dataset


In [8]:
# 🚀 ENHANCED MOBILENETV2 (Fine-tuning approach)
def create_enhanced_mobilenet():
    """MobileNetV2 with fine-tuning for small datasets"""
    # Load pre-trained MobileNetV2
    base_model = tf.keras.applications.MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # Unfreeze top layers for fine-tuning (better for small datasets)
    base_model.trainable = True
    fine_tune_at = len(base_model.layers) - 30  # Unfreeze last 30 layers
    
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    # Add custom top
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model

# Create enhanced MobileNet
mobilenet = create_enhanced_mobilenet()

print("🚀 Enhanced MobileNetV2 created with:")
print("   ✅ Fine-tuning enabled (last 30 layers)")
print("   ✅ Custom regularized head")
print("   ✅ Optimized for small dataset")


🚀 Enhanced MobileNetV2 created with:
   ✅ Fine-tuning enabled (last 30 layers)
   ✅ Custom regularized head
   ✅ Optimized for small dataset


In [9]:
# ⚙️ COMPILATION & TRAINING

# Create an optimizer for the Custom CNN
optimizer_cnn = Adam(learning_rate=0.0001, weight_decay=1e-4)

# Create a separate, new optimizer for MobileNetV2
optimizer_mobilenet = Adam(learning_rate=0.0001, weight_decay=1e-4)

# Compile the models with their respective optimizers
custom_cnn.compile(optimizer=optimizer_cnn, loss='categorical_crossentropy', metrics=['accuracy'])
mobilenet.compile(optimizer=optimizer_mobilenet, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks can be shared
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-7, verbose=1)
]

print("⚙️ Models compiled with optimized settings for small dataset")
print("📞 Callbacks configured with strong early stopping")

# 🏋️ TRAIN MODELS
print("\n🏋️ Training Custom CNN...")
history_cnn = custom_cnn.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n🏋️ Training Enhanced MobileNetV2...")
history_mobilenet = mobilenet.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training completed!")

⚙️ Models compiled with optimized settings for small dataset
📞 Callbacks configured with strong early stopping

🏋️ Training Custom CNN...
Epoch 1/50
30/30 [==============================] - 40s 1s/step - loss: 2.3119 - accuracy: 0.1000 - val_loss: 2.3032 - val_accuracy: 0.1000 - lr: 1.0000e-04
Epoch 2/50
30/30 [==============================] - 35s 1s/step - loss: 2.2869 - accuracy: 0.1354 - val_loss: 2.3050 - val_accuracy: 0.1000 - lr: 1.0000e-04
Epoch 3/50
30/30 [==============================] - 36s 1s/step - loss: 2.2529 - accuracy: 0.1792 - val_loss: 2.3061 - val_accuracy: 0.1000 - lr: 1.0000e-04
Epoch 4/50
30/30 [==============================] - 39s 1s/step - loss: 2.1488 - accuracy: 0.1896 - val_loss: 2.3068 - val_accuracy: 0.1000 - lr: 1.0000e-04
Epoch 5/50
30/30 [==============================] - 39s 1s/step - loss: 2.0306 - accuracy: 0.2354 - val_loss: 2.3265 - val_accuracy: 0.1000 - lr: 1.0000e-04
Epoch 6/50
30/30 [==============================] - ETA: 0s - loss: 1.9659 - 

In [11]:
# 🎭 Create and evaluate ensemble
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
pred1 = custom_cnn(inputs)
pred2 = mobilenet(inputs)
ensemble_output = tf.keras.layers.Average()([pred1, pred2])
ensemble_model = Model(inputs=inputs, outputs=ensemble_output)

# Compile with explicit optimizer
ensemble_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, weight_decay=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Evaluate ensemble
ensemble_scores = ensemble_model.evaluate(validation_generator, verbose=0)
print(f"\n🎭 ENSEMBLE MODEL Results:")
print(f"   📉 Validation Loss: {ensemble_scores[0]:.4f}")
print(f"   🎯 Validation Accuracy: {ensemble_scores[1]:.4f} ({ensemble_scores[1]*100:.2f}%)")

# 💾 Saving trained models...
print(f"\n💾 Saving trained models...")
custom_cnn.save('new_custom_cnn.h5')
mobilenet.save('new_mobilenet.h5')

print(f"✅ Models saved:")
print(f"   📁 new_custom_cnn.h5")
print(f"   📁 new_mobilenet.h5")
print(f"   🎭 Ensemble: Will be recreated in app.py")

print(f"\n🎉 TRAINING COMPLETE!")
print(f"📊 Best performance: {ensemble_scores[1]*100:.2f}%")
print(f"🚀 Ready for deployment!")


🎭 ENSEMBLE MODEL Results:
   📉 Validation Loss: 0.6378
   🎯 Validation Accuracy: 0.9833 (98.33%)

💾 Saving trained models...
✅ Models saved:
   📁 new_custom_cnn.h5
   📁 new_mobilenet.h5
   🎭 Ensemble: Will be recreated in app.py

🎉 TRAINING COMPLETE!
📊 Best performance: 98.33%
🚀 Ready for deployment!
